# Описание модели

## Постановка задачи и использованные данные

**БТ**:

Во время обслуживания клиентов сотрудники КЦ пользуются справочными документами - так называемыми интерактивными схемами (ИС) - на портале Knowledge Assistant. Операторы не всегда точно знают, в какой схеме содержится ответ на вопрос клиента, в связи с чем возникает потребность автоматического поиска соответствующего документа. Для решения этой задачи разрабатывается модель оценки семантической близости между запросом и документом.

**Данные**:

Построение корпуса поисковых запросов и их разметка на документы в процессе, поэтому для создания прототипа поисковой модели использовались данные открытого датасета парафраз, который состоит из новостных заголовков и их перефразировок. Всего в корпусе 1998622 (~$2 \cdot 10^6$) уникальных текста.

## Подготовка данных

Как уже было упомянуто ранее, для обучения прототипа поисковой модели использовался открытый датасет парафраз новостных заголовков.  

Из пар текстов и их перефразировок был сформирован корпус триплетов:
- anchor - "опорный" текст
- positive - положительный пример парафразы anchor-а
- negative - отрицательный пример парафразы anchor-а (выбирается случайно из множества не связанных с anchor-ом текстов)

## Формирование выборок

Датасет триплетов разбит на train, val и test выборки в соотношении 70% / 20% / 10%

## Описание модели

Для ранжирования текстов-"документов" относительно текста-"запроса" используется косинусное расстояние между векторными представлениями этих текстов в качестве оценки семантической близости.

### Обучение векторных представлений текстов

Архитектура модели-энкодера представляет собой сиамскую нейронную сеть с тремя входами для триплета "anchor - positive - negative".

Каждый из входов разбивается на токены (токенизатором из библиотеки Keras), затем последовательность токенов индексируется по словарю, после чего кодируется модулями Embedding и BiLSTM (таблица Embedding инициализируется предобученными векторами FastText). На векторизированном таким образом триплете вычисляется softmax-loss для обучения параметров энкодера. 

<img src="model_architecture.png">

### Валидация

Для оценки векторов, которые модель отдаёт на выходе в качестве эмбеддингов, считается косинусное расстояние между векторами пар "anchor - positive" и "anchor - negative", а затем вычисляется корреляция этого сходства с истинными значениями.

В качестве метрики используется ранговая корреляция Пирсона и Спирмена с метками 1 и -1: считаем, что эмбеддинги положительной пары априори сонаправленные, между ними угол ноль, а эмбеддинги отрицательной - противоположные, между ними угол $\pi$.   

### Метрика оценки качества ранжирования

Для оценки качества ранжирования используется метрика $MRR@K$ - среднеобратный ранг на топ-$K$ отобранных объектов для заданного. $K = 20$, так как на портале Knowledge Assistant для поискового запроса отображается 20 результатов (остальные документы подгружаются "по требованию").

Был выбран именно $MRR$, так как он учитывает позицию первого верно предсказанного элемента в отранжированном списке. Оператору важно, чтобы документ, который содержит ответ на запрос, был как можно выше в поисковой выдаче. 

$$MRR@K = \frac{1}{N}\displaystyle\sum_{j=1}^{N}RR@K_j$$

$$RR@K = \frac{1}{min\{k \in [1 \dots K]: r^{true}(\pi^{-1}(k)) = 1\}}$$

## Производительность

* Среднее время построения вектора одного текста на CPU составляет **23.05 мс**. 

* Размер модели энкодера составляет **28.9 Mb**.

## Проверка качества модели

### Качество модели на отложенной выборке

Значение $MRR@20$ на тестовой выборке: $0.45$